In [ ]:
# This model is used to predict the flood risk of a location based on regression model (XGBoostregressor)

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBRegressor
from flood_tool.geo import get_gps_lat_long_from_easting_northing
from sklearn.metrics import mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint


In [2]:
data = pd.read_csv('flood_tool/resources/postcodes_labelled.csv')

In [3]:
data.drop_duplicates(inplace=True)

coordinates_lat = get_gps_lat_long_from_easting_northing(data['easting'], data['northing'])
coordinates_df = pd.DataFrame({
    'Latitude': coordinates_lat[0],
    'Longitude': coordinates_lat[1]
})
data = pd.concat([data, coordinates_df], axis=1)
data.drop(columns=['easting', 'northing'], inplace=True)

data['log_medianPrice'] = np.log(data['medianPrice'])

# Create bins for log-transformed medianPrice based on the description of log median price
bins = [-np.inf, 11.9, 14.2, np.inf]
labels = ['Low', 'Medium', 'High']

# Apply log transformation and binning
data['price_category'] = pd.cut(data['log_medianPrice'], bins=bins, labels=labels)
# Display the first few rows to verify
data[['medianPrice', 'log_medianPrice', 'price_category']].head()
# postcodes_labelled.isnull().sum()

#data['nearestWatercourse'].nunique() # 1146 + Nan which is No nearest WaterCourse
data['nearestWatercourse'].fillna('NoStreamNear', inplace=True)

## Define bins and labels
elevation_bins = [-10, 0, 40, 80, np.inf]
elevation_labels = ['BelowSeaLevel', 'Low', 'Medium', 'High']
# Apply binning to the elevation column
data['elevation_category'] = pd.cut(data['elevation'], bins=elevation_bins, labels=elevation_labels)

# Define bins and labels
distance_bins = [0, 630, 1090, 1840, np.inf]
distance_labels = ['Very Close', 'Close', 'Moderate', 'Far']

# Apply binning to the distanceToWatercourse column
data['distanceToWatercourse_category'] = pd.cut(data['distanceToWatercourse'], bins=distance_bins, labels=distance_labels)

bins = [-np.inf, 11.9, 14.2, np.inf]
labels = ['Low', 'Medium', 'High']

# Apply log transformation and binning
data['log_medianPrice'] = np.log(data['medianPrice'])
data['price_category'] = pd.cut(data['log_medianPrice'], bins=bins, labels=labels)

/var/folders/2k/qf5v72y97y709qmzdz43p2f00000gn/T/ipykernel_51967/3881437295.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['nearestWatercourse'].fillna('NoStreamNear', inplace=True)


In [4]:
def log_transform(x):
    return np.log1p(x+np.abs(data['elevation'].min()))

numeric_features = ['elevation', 'distanceToWatercourse', 'soilType_encoded', 'nearestWatercourse_encoded', 'localAuthority_encoded']
numeric_remained = ['Latitude', 'Longitude']
categorical_features = []

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log_transform', FunctionTransformer(log_transform, validate=True)),
    ('scaler', RobustScaler())
])

numeric_remained_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('num_rem', numeric_remained_transformer, numeric_remained),
        ('cat', categorical_transformer, categorical_features)
    ])

standard_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [5]:
# labelEncode (outside of pipeline)
label_encoder = LabelEncoder()
data['localAuthority_encoded'] = label_encoder.fit_transform(data['localAuthority'])
data['soilType_encoded'] = label_encoder.fit_transform(data['soilType'])
data['nearestWatercourse_encoded'] = label_encoder.fit_transform(data['nearestWatercourse'])

X = data[['elevation', 'soilType_encoded', 'nearestWatercourse_encoded', 'distanceToWatercourse', 'Latitude', 'Longitude', 'localAuthority_encoded']]
y = data['riskLabel']

In [6]:
# enconde y to 0, 1, 2, 3, 4, 5, 6
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

X_train_transformed = standard_pipeline.fit_transform(X_train)
X_test_transformed = standard_pipeline.transform(X_test)

# imbalanced data handling (SMOTE and ADASYN)
# SMOTE: Generate a few class samples by interpolation.
# ADASYN: Add noise around a few class samples to produce a more diverse sample.
smote = SMOTE(sampling_strategy={5: 5000, 6: 2000}, random_state=42)
adasyn = ADASYN(sampling_strategy={5: 6000, 6: 3000}, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_resampled, y_train_resampled)

# data augmentation for class 6 and 7
X_train_class6 = X_train_transformed[y_train == 5]
X_train_class7 = X_train_transformed[y_train == 6]
noise6 = np.random.normal(0, 0.01, X_train_class6.shape)
noise7 = np.random.normal(0, 0.01, X_train_class7.shape)
X_augmented = np.vstack([X_train_class6 + noise6, X_train_class7 + noise7])
y_augmented = np.hstack([np.full(X_train_class6.shape[0], 5), np.full(X_train_class7.shape[0], 6)])

X_train_resampled = np.vstack([X_train_resampled, X_augmented])
y_train_resampled = np.hstack([y_train_resampled, y_augmented])

# calculate class weights based on the quantity of samples
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

XGboost = XGBRegressor(
    random_state=42,
    n_estimators=300,
    max_depth=12,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss'
)
XGboost.fit(X_train_resampled, y_train_resampled)

# predict the test set
y_pred_continuous = XGboost.predict(X_test_transformed)

# round the continuous predictions to the nearest integer
y_pred_encoded = np.round(y_pred_continuous).astype(int)

# make sure the predictions are within the range of the classes
y_pred_encoded = np.clip(y_pred_encoded, 0, len(label_encoder.classes_) - 1)

# decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)
y_test_original = label_encoder.inverse_transform(y_test)

# evaluate the model
mse = mean_squared_error(y_test_original, y_pred)  # MSE: Mean Squared Error
rmse = np.sqrt(mse)  # RMSE: Root Mean Squared Error
r2 = r2_score(y_test_original, y_pred)  # R^2: R-squared

# print results
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R-squared (R²): {r2:.4f}')

Mean Squared Error (MSE): 0.8547
Root Mean Squared Error (RMSE): 0.9245
R-squared (R²): 0.3800


In [7]:
# define the hyperparameter search space
param_dist = {
    'n_estimators': randint(100, 500),       # 树的数量
    'max_depth': randint(3, 15),             # 树的最大深度
    'learning_rate': uniform(0.01, 0.3),    # 学习率
    'subsample': uniform(0.5, 0.5),         # 随机采样比例
    'colsample_bytree': uniform(0.5, 0.5),  # 每棵树使用的特征比例
    'reg_alpha': uniform(0, 10),            # L1 正则化
    'reg_lambda': uniform(0, 10)            # L2 正则化
}

# create an XGBoost model
XGboost = XGBRegressor(random_state=42, eval_metric='rmse')

# define the random search
random_search = RandomizedSearchCV(
    estimator=XGboost,
    param_distributions=param_dist,
    n_iter=50,  # 随机搜索迭代次数
    scoring='neg_mean_squared_error',  # 使用负均方误差作为得分
    cv=5,  # 交叉验证折数
    verbose=0,  # 显示搜索过程
    random_state=42,
    n_jobs=-1  # 使用所有可用 CPU 进行并行计算
)

# conduct the random search
random_search.fit(X_train_resampled, y_train_resampled)

# Get the best parameters
best_params = random_search.best_params_
print(f"Best parameters: {best_params}")

# use the best model to make predictions
best_model = random_search.best_estimator_
y_pred_continuous = best_model.predict(X_test_transformed)

# round the continuous predictions to the nearest integer
y_pred_encoded = np.round(y_pred_continuous).astype(int)

# make sure the predictions are within the range of the classes
y_pred_encoded = np.clip(y_pred_encoded, 0, len(label_encoder.classes_) - 1)

# decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)
y_test_original = label_encoder.inverse_transform(y_test)

# evaluate the model
mse = mean_squared_error(y_test_original, y_pred)  # MSE: Mean Squared Error
rmse = np.sqrt(mse)  # RMSE: Root Mean Squared Error
r2 = r2_score(y_test_original, y_pred)  # R^2: R-squared

# print results
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R-squared (R²): {r2:.4f}')

/opt/anaconda3/envs/npp2024/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'colsample_bytree': 0.9050566973395904, 'learning_rate': 0.2701216955740311, 'max_depth': 14, 'n_estimators': 490, 'reg_alpha': 5.015162946871996, 'reg_lambda': 7.9829517896677515, 'subsample': 0.8249819653888826}
Mean Squared Error (MSE): 0.8947
Root Mean Squared Error (RMSE): 0.9459
R-squared (R²): 0.3511
